# Assignment 1

In this assignment, you will implement a multi-layer perceptron (MLP) using PyTorch.


## Task 1

In [6]:
def f(x):
    return 3 * x ** 2 - 4 * x + 5

def gradient(f, x, h=0.0001):
    return (f(x + h) - f(x)) / h

gradient(f, 2)

8.000300000023941

In [7]:
# What happens if we keep decreasing h?
gradient(f, 2, h=0.0000000000000001)

# Why is the gradient 0?
# If you don't know, google it!
# ------------------
# Write your implementation here.
# ANSWER:

# ------------------

0.0

## Task 2

Predict housing prices using linear regression.

In [15]:
import os
import pandas as pd
root_dir = os.getcwd() + "/../../"
melbourne_data = pd.read_csv(root_dir + "data/raw/melb_data.csv")
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']

In [16]:
# Input data
X = melbourne_data[melbourne_features]
X

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
0,2,1.0,202.0,-37.79960,144.99840
1,2,1.0,156.0,-37.80790,144.99340
2,3,2.0,134.0,-37.80930,144.99440
3,3,2.0,94.0,-37.79690,144.99690
4,4,1.0,120.0,-37.80720,144.99410
...,...,...,...,...,...
13575,4,2.0,652.0,-37.90562,145.16761
13576,3,2.0,333.0,-37.85927,144.87904
13577,3,2.0,436.0,-37.85274,144.88738
13578,4,1.0,866.0,-37.85908,144.89299


In [17]:
# Target data
y = melbourne_data.Price
y

0        1480000.0
1        1035000.0
2        1465000.0
3         850000.0
4        1600000.0
           ...    
13575    1245000.0
13576    1031000.0
13577    1170000.0
13578    2500000.0
13579    1285000.0
Name: Price, Length: 13580, dtype: float64

### Model

In [ ]:
# ------------------
# Write your implementation here.
# or you can use PyTorch nn.Module
class LinearRegression:
    def __init__(self):
        self.weights = None
        self.bias = None

    def __call__(self, x):
        """
        Forward pass
        y = Wx + b
        """

    def parameters(self):
        return self.weights + [self.bias]
# ------------------

### Training

In [ ]:
# Learning rate
lr = 0.001

# Training loop
for step in range(10):
    # ------------------
    # Write your implementation here.
    pass
    # ------------------

## Extra Credit

Linear regression is the simplest form of neural networks. It actually does not require gradient descent to solve for the weights.

**Find a way to get the weights of the linear regression model without using gradient descent.**

Apply your solution on the Melbourne housing dataset.

In [ ]:
# ------------------
# Write your implementation here.
# ------------------